# Optimizing Model Parameters

Imports

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Prerequiste code

In [2]:
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

print(f"device: {device}")

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)

device: cuda


Hyperparameters

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

Loss function

In [4]:
loss_fn = nn.CrossEntropyLoss()

Optimizer

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Full Implementation

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)

  model.train()

  for batch, (X, y) in enumerate(dataloader):
    X = X.to(device)
    y = y.to(device)

    # Compute the prediction and loss
    pred = model(X)
    loss = loss_fn(pred, y)

    # Perform backprop
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:7f} [{current:>5d}/{size:5d}]")

In [7]:
def test_loop(dataloader, model, loss_fn):
  model.eval()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      X = X.to(device)
      y = y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()


  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy {100 * correct:>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
epochs = 10
for t in range(epochs):
  print(f"Epoch: {t+1}\n---------------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)

print("Done!")

Epoch: 1
---------------------------------
loss: 2.291566 [   64/60000]
loss: 2.283365 [ 6464/60000]
loss: 2.263372 [12864/60000]
loss: 2.265101 [19264/60000]
loss: 2.243771 [25664/60000]
loss: 2.213560 [32064/60000]
loss: 2.230381 [38464/60000]
loss: 2.189734 [44864/60000]
loss: 2.185502 [51264/60000]
loss: 2.162635 [57664/60000]
Test Error: 
 Accuracy 37.7%, Avg loss: 2.151333 

Epoch: 2
---------------------------------
loss: 2.155950 [   64/60000]
loss: 2.145490 [ 6464/60000]
loss: 2.088720 [12864/60000]
loss: 2.114532 [19264/60000]
loss: 2.065638 [25664/60000]
loss: 1.998651 [32064/60000]
loss: 2.038881 [38464/60000]
loss: 1.953502 [44864/60000]
loss: 1.966091 [51264/60000]
loss: 1.903831 [57664/60000]
Test Error: 
 Accuracy 54.6%, Avg loss: 1.891538 

Epoch: 3
---------------------------------
loss: 1.917212 [   64/60000]
loss: 1.886294 [ 6464/60000]
loss: 1.772053 [12864/60000]
loss: 1.826260 [19264/60000]
loss: 1.720037 [25664/60000]
loss: 1.660085 [32064/60000]
loss: 1.694009 